## CREATE A VECTOR DB

1. Create a list of articles (link, date)

In [1]:
keywords = [
    # Energy Sector
    "XOM", "CVX", "COP", "EOG", "SLB",
    "Exxon", "Chevron", "ConocoPhillips", "EOG Resources Inc", "Schlumberger NV",
    "oil", "gas", "energy", "OPEC", "power",
    "electricity", "green", "utilities",

    # Financials Sector
    "JPM", "BAC", "WFC", "AXP", "MS",
    "JPMorgan", "Bank of America", "Wells Fargo", "American Express", "Morgan Stanley",
    "bank", "interest rates", "savings", "investment", "regulation",
    "inflation", "employment", "stock", "bond", "FED",
    "SEC", "NYSE", "NASDAQ", "S&P500",

    # Tech Sector
    "MSFT", "AAPL", "NVDA", "GOOGL", "META",
    "Microsoft", "Apple", "Nvidia", "Alphabet", "Meta",
    "AI", "Google", "cybersecurity", "fintech", "data", "cloud"
]

In [27]:
# ft articles data [heading, link, date]

import requests
from bs4 import BeautifulSoup
from datetime import datetime


news_data = []
keyword = 'aapl'
cutoff_date = datetime.strptime("01-Dec-2018", "%d-%b-%Y")

keyurl = f"https://www.ft.com/search?q={keyword}&page=1&sort=date&isFirstView=true"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}

keyresult = requests.get(keyurl, headers=headers)
# keyresult = requests.get(keyurl)
print(f"HTTP Status Code: {keyresult.status_code}")

keyresult_content = keyresult.content
keysoup = BeautifulSoup(keyresult_content, "html.parser")
# print(keysoup.prettify())
maxpages = keysoup.find('span', {"class": "search-pagination__page"})
maxpages = int(maxpages.get_text(strip=True).split()[-1])
# maxpages += 5
print(maxpages)


for page in range(1, maxpages):
    url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
    result = requests.get(url, headers=headers)
    # print(page)
    # print(result.text)
    if "Sorry" in result.text: # maybe unnecessary
        print(f"Stopping iteration: Page {page} is not available.")
        break
    
    result_content = result.content
    soup = BeautifulSoup(result_content, "lxml")
    
    for article in soup.findAll("div", {"class": "o-teaser"}):
        heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True) #find(text=True), find text is outdated
        link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
        date = article.find('time', {"class": "o-teaser__timestamp-date"})
        
        # print(heading, link, date)
        if heading and link and date:
            article_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
            if article_date > cutoff_date:
                news_data.append([heading, link['href'], date.text.strip()])

# print(pages)

print(f"Total articles collected in news_data: {len(news_data)}")
print(news_data)

HTTP Status Code: 200
2
Total articles collected in news_data: 7
[['Is ChatGPT good at day trading? Probably not', '/content/0608b6a0-f586-430c-9e68-a177c544435a', 'March 28, 2024'], ['GraniteShares files to increase leverage on 17 ETFs', '/content/f26c4b3b-baa4-4586-9fa2-bf8d63fd457b', 'November 30, 2023'], ['Microsoft, and a macro lesson', '/content/d5ba4cab-7976-4352-b624-7654c6acf7c3', 'October 28, 2021'], ['High expectations leave no room for errorPremiumcontent', '/content/ed5c04dd-75cf-4338-b51c-869affebedde', 'July 7, 2020'], ['Markets not live, Thursday 30th January 2020', '/content/ad45f2d9-07e6-4b0c-a040-f831f8af5ae8', 'January 30, 2020'], ['China’s credit pulse has a kickPremiumcontent', '/content/3c35f688-36f6-11ea-a6d3-9a26f8c3cba4', 'January 14, 2020'], ['Investors Chronicle: Tesco, Serco, Apple', '/content/6fbc606c-1359-11e9-a581-4ff78404524e', 'January 11, 2019']]
